# Gold Meta-Model Training - Attempt 19

**Architecture:** Optuna HPO (100 trials) -> Bootstrap Data Subsampling Ensemble (12 models, 80% bootstrap)

**Key Changes from Attempt 18:**
1. **Optuna HPO (100 trials)** to find optimal hyperparameters for 28-feature space
   - Attempt 18 used attempt 7 params optimized for 24 features, NOT 28 features
   - Root cause of regression: mismatched feature space and hyperparameters
2. **Composite objective** for HPO: DA * 0.6 + Sharpe_norm * 0.3 + (-MAE_norm) * 0.1
3. **Bootstrap ensemble** (12 models, 80%) applied AFTER HPO with best params

**Feature Set (28 features):**
- Base features (5): real_rate_change, dxy_change, vix, yield_spread_change, inflation_exp_change
- VIX submodel (3): vix_regime_probability, vix_mean_reversion_z, vix_persistence
- Technical submodel (3): tech_trend_regime_prob, tech_mean_reversion_z, tech_volatility_regime
- Cross-asset submodel (3): xasset_regime_prob, xasset_recession_signal, xasset_divergence
- Yield curve submodel (2): yc_spread_velocity_z, yc_curvature_z
- ETF flow submodel (3): etf_regime_prob, etf_capital_intensity, etf_pv_divergence
- Inflation expectation submodel (3): ie_regime_prob, ie_anchoring_z, ie_gold_sensitivity_z
- Options market submodel (1): options_risk_regime_prob
- Temporal context submodel (1): temporal_context_score
- Real rate submodel attempt 7 (4): rr_level_change_z, rr_slope_chg_z, rr_curvature_chg_z, rr_slope_level_z

**Reference Metrics:**
- Attempt 7 (best overall): DA 60.04%, HCDA 64.13%, MAE 0.9429%, Sharpe 2.4636
- Attempt 18 (predecessor): DA 58.30%, HCDA 63.04%, MAE 0.9527%, Sharpe 1.8624
- Design: Optuna HPO specifically tuned for 28-feature space

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
import json
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
np.random.seed(42)

print(f"XGBoost version: {xgb.__version__}")
print(f"Optuna version: {optuna.__version__}")
print(f"Started: {datetime.now().isoformat()}")
print(f"Attempt: 19")
print(f"Architecture: Optuna HPO (100 trials) + Bootstrap Data Subsampling Ensemble (12 models, 80%)")

## Feature Definitions (28 features)

In [ ]:
FEATURE_COLUMNS = [
    # Base features (5)
    'real_rate_change',
    'dxy_change',
    'vix',
    'yield_spread_change',
    'inflation_exp_change',
    # VIX submodel (3)
    'vix_regime_probability',
    'vix_mean_reversion_z',
    'vix_persistence',
    # Technical submodel (3)
    'tech_trend_regime_prob',
    'tech_mean_reversion_z',
    'tech_volatility_regime',
    # Cross-asset submodel (3)
    'xasset_regime_prob',
    'xasset_recession_signal',
    'xasset_divergence',
    # Yield curve submodel (2)
    'yc_spread_velocity_z',
    'yc_curvature_z',
    # ETF flow submodel (3)
    'etf_regime_prob',
    'etf_capital_intensity',
    'etf_pv_divergence',
    # Inflation expectation submodel (3)
    'ie_regime_prob',
    'ie_anchoring_z',
    'ie_gold_sensitivity_z',
    # Options market submodel (1)
    'options_risk_regime_prob',
    # Temporal context submodel (1)
    'temporal_context_score',
    # Real rate submodel attempt 7 (4)
    'rr_level_change_z',
    'rr_slope_chg_z',
    'rr_curvature_chg_z',
    'rr_slope_level_z',
]

TARGET = 'gold_return_next'

assert len(FEATURE_COLUMNS) == 28, f"Expected 28 features, got {len(FEATURE_COLUMNS)}"
print(f"Features defined: {len(FEATURE_COLUMNS)} features")

## Loading Data from Kaggle Dataset (no API key needed)

In [ ]:
print("="*60)
print("LOADING DATA FROM KAGGLE DATASET (no API key needed)")
print("="*60)

# Dataset path resolution (inline - needed before main cell-5 resolution)
_PROBE_FILE_BF = 'base_features_raw.csv'
_glob_patterns_bf = [
    f'/kaggle/input/datasets/bigbigzabuton/gold-prediction-submodels/{_PROBE_FILE_BF}',
    f'/kaggle/input/gold-prediction-submodels/{_PROBE_FILE_BF}',
    f'/kaggle/input/datasets/*/gold-prediction-submodels/{_PROBE_FILE_BF}',
    f'/kaggle/input/*/{_PROBE_FILE_BF}',
]

DATASET_BASE_EARLY = None
for _pattern in _glob_patterns_bf:
    _matches = glob.glob(_pattern)
    if _matches:
        _candidate_base = os.path.dirname(_matches[0])
        try:
            pd.read_csv(_matches[0], nrows=1)
            DATASET_BASE_EARLY = _candidate_base
            print(f"Dataset found at: {DATASET_BASE_EARLY} (pattern: {_pattern})")
            break
        except Exception as _e:
            print(f"  Found at {_matches[0]} but read failed: {_e}")

if DATASET_BASE_EARLY is None:
    print("ERROR: base_features_raw.csv not found! Searched:")
    for _p in _glob_patterns_bf:
        print(f"  {_p} -> {glob.glob(_p)}")
    try:
        import subprocess as _sp
        _r = _sp.run(['find', '/kaggle/input', '-name', _PROBE_FILE_BF, '-type', 'f'],
                     capture_output=True, text=True, timeout=15)
        print(f"  find result: {_r.stdout.strip() or '(nothing found)'}")
    except Exception as _e:
        print(f"  find failed: {_e}")
    raise FileNotFoundError(
        "base_features_raw.csv not found in gold-prediction-submodels dataset. "
        "Ensure dataset was updated with base_features_raw.csv."
    )

# Load base features (gold price, FRED series) from pre-fetched dataset
print("\nLoading base_features_raw.csv from dataset...")
bf = pd.read_csv(f'{DATASET_BASE_EARLY}/base_features_raw.csv')
bf['Date'] = pd.to_datetime(bf['Date']).dt.strftime('%Y-%m-%d')
bf = bf.set_index('Date')
print(f"  Loaded: {len(bf)} rows, {bf.index.min()} to {bf.index.max()}")
print(f"  Columns: {list(bf.columns)}")

# Build base_features with same structure as original pipeline
base_features = bf[['gold_return_next', 'real_rate_real_rate', 'dxy_dxy', 'vix_vix',
                     'yield_curve_yield_spread',
                     'inflation_expectation_inflation_expectation']].copy()
base_features = base_features.ffill()
base_features = base_features.dropna(subset=['gold_return_next'])
print(f"  Base features: {len(base_features)} rows, {len(base_features.columns)} columns")
print("\nData loading complete")

## Feature Transformation and Submodel Loading

In [ ]:
# ============================================================
# DATASET PATH RESOLUTION (robust: use glob + pd.read_csv probe)
# API-created kernels: /kaggle/input/datasets/{owner}/{slug}/
# Web-UI-created kernels: /kaggle/input/{slug}/
# Use glob to find the file, then probe with pd.read_csv to verify
# ============================================================
import pandas as _pd_probe
DATASET_SLUG = 'gold-prediction-submodels'
DATASET_OWNER = 'bigbigzabuton'
_PROBE_FILE = 'vix.csv'

# Use glob to search all plausible locations with absolute paths
_glob_patterns = [
    f'/kaggle/input/datasets/{DATASET_OWNER}/{DATASET_SLUG}/{_PROBE_FILE}',
    f'/kaggle/input/{DATASET_SLUG}/{_PROBE_FILE}',
    f'/kaggle/input/datasets/*/{DATASET_SLUG}/{_PROBE_FILE}',
    f'/kaggle/input/*/{_PROBE_FILE}',
]

DATASET_BASE = None
for _pattern in _glob_patterns:
    _matches = glob.glob(_pattern)
    if _matches:
        _candidate_base = os.path.dirname(_matches[0])
        # Verify by actually reading the file with pandas
        try:
            _pd_probe.read_csv(_matches[0], nrows=1)
            DATASET_BASE = _candidate_base
            print(f"Dataset found at: {DATASET_BASE} (pattern: {_pattern})")
            break
        except Exception as _e:
            print(f"  Found at {_matches[0]} but read failed: {_e}")

if DATASET_BASE is None:
    print("ERROR: Dataset not found via glob! Searched patterns:")
    for _p in _glob_patterns:
        print(f"  {_p} -> {glob.glob(_p)}")
    print("\nFull /kaggle/input/ listing:")
    try:
        import subprocess as _sp
        _r = _sp.run(['find', '/kaggle/input', '-name', _PROBE_FILE, '-type', 'f'],
                     capture_output=True, text=True, timeout=15)
        print(f"  find result: {_r.stdout.strip() or '(nothing found)'}")
        print(f"  find stderr: {_r.stderr.strip()}")
    except Exception as _e:
        print(f"  find failed: {_e}")
    raise FileNotFoundError(
        f"Dataset '{DATASET_SLUG}' not found. "
        f"Tried patterns: {_glob_patterns}. "
        "Ensure dataset_sources includes 'bigbigzabuton/gold-prediction-submodels' in kernel-metadata.json."
    )

print("\nApplying transformations...")

final_df = base_features.copy()

final_df['real_rate_change'] = final_df['real_rate_real_rate'].diff()
final_df['dxy_change'] = final_df['dxy_dxy'].diff()
final_df['vix'] = final_df['vix_vix']
final_df['yield_spread_change'] = final_df['yield_curve_yield_spread'].diff()
final_df['inflation_exp_change'] = final_df['inflation_expectation_inflation_expectation'].diff()

final_df = final_df.drop(columns=['real_rate_real_rate', 'dxy_dxy', 'vix_vix',
                                    'yield_curve_yield_spread', 'inflation_expectation_inflation_expectation'])

print(f"  Base transformations applied")
print(f"  Columns so far: {list(final_df.columns)}")

print("\nLoading submodel outputs from Kaggle Dataset...")

submodel_files = {
    'vix': {
        'path': f'{DATASET_BASE}/vix.csv',
        'columns': ['vix_regime_probability', 'vix_mean_reversion_z', 'vix_persistence'],
        'date_col': 'date',
        'tz_aware': False,
    },
    'technical': {
        'path': f'{DATASET_BASE}/technical.csv',
        'columns': ['tech_trend_regime_prob', 'tech_mean_reversion_z', 'tech_volatility_regime'],
        'date_col': 'date',
        'tz_aware': True,
    },
    'cross_asset': {
        'path': f'{DATASET_BASE}/cross_asset.csv',
        'columns': ['xasset_regime_prob', 'xasset_recession_signal', 'xasset_divergence'],
        'date_col': 'Date',
        'tz_aware': False,
    },
    'yield_curve': {
        'path': f'{DATASET_BASE}/yield_curve.csv',
        'columns': ['yc_spread_velocity_z', 'yc_curvature_z'],
        'date_col': 'index',
        'tz_aware': False,
    },
    'etf_flow': {
        'path': f'{DATASET_BASE}/etf_flow.csv',
        'columns': ['etf_regime_prob', 'etf_capital_intensity', 'etf_pv_divergence'],
        'date_col': 'Date',
        'tz_aware': False,
    },
    'inflation_expectation': {
        'path': f'{DATASET_BASE}/inflation_expectation.csv',
        'columns': ['ie_regime_prob', 'ie_anchoring_z', 'ie_gold_sensitivity_z'],
        'date_col': 'Unnamed: 0',
        'tz_aware': False,
    },
    'options_market': {
        'path': f'{DATASET_BASE}/options_market.csv',
        'columns': ['options_risk_regime_prob'],
        'date_col': 'Date',
        'tz_aware': True,
    },
    'temporal_context': {
        'path': f'{DATASET_BASE}/temporal_context.csv',
        'columns': ['temporal_context_score'],
        'date_col': 'date',
        'tz_aware': False,
    },
    'real_rate': {
        'path': f'{DATASET_BASE}/real_rate.csv',
        'columns': ['rr_level_change_z', 'rr_slope_chg_z', 'rr_curvature_chg_z', 'rr_slope_level_z'],
        'date_col': 'date',
        'tz_aware': False,
    },
}

submodel_dfs = {}
for feature, spec in submodel_files.items():
    df = pd.read_csv(spec['path'])
    date_col = spec['date_col']
    if spec['tz_aware']:
        df['Date'] = pd.to_datetime(df[date_col], utc=True).dt.strftime('%Y-%m-%d')
    else:
        if date_col == 'index':
            df['Date'] = pd.to_datetime(df.iloc[:, 0]).dt.strftime('%Y-%m-%d')
        elif date_col == 'Unnamed: 0':
            df['Date'] = pd.to_datetime(df['Unnamed: 0']).dt.strftime('%Y-%m-%d')
        else:
            df['Date'] = pd.to_datetime(df[date_col]).dt.strftime('%Y-%m-%d')
    df = df[['Date'] + spec['columns']]
    df = df.set_index('Date')
    submodel_dfs[feature] = df
    print(f"  {feature}: {len(df)} rows")

print("\nMerging submodel outputs...")
for feature, df in submodel_dfs.items():
    final_df = final_df.join(df, how='left')
print(f"  Features after merge: {final_df.shape[1]} columns, {len(final_df)} rows")

print("\nApplying NaN imputation...")
nan_before = final_df.isna().sum().sum()
print(f"  NaN before imputation: {nan_before}")

regime_cols = ['vix_regime_probability', 'tech_trend_regime_prob',
               'xasset_regime_prob', 'etf_regime_prob', 'ie_regime_prob',
               'options_risk_regime_prob', 'temporal_context_score']
for col in regime_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.5)

z_cols = ['vix_mean_reversion_z', 'tech_mean_reversion_z',
          'yc_spread_velocity_z', 'yc_curvature_z',
          'etf_capital_intensity', 'etf_pv_divergence',
          'ie_anchoring_z', 'ie_gold_sensitivity_z']
for col in z_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.0)

div_cols = ['xasset_recession_signal', 'xasset_divergence']
for col in div_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.0)

cont_cols = ['tech_volatility_regime', 'vix_persistence']
for col in cont_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(final_df[col].median())

# Real rate submodel z-scores: fill with 0.0 (neutral / no signal)
rr_z_cols = ['rr_level_change_z', 'rr_slope_chg_z', 'rr_curvature_chg_z', 'rr_slope_level_z']
for col in rr_z_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].fillna(0.0)

final_df = final_df.dropna(subset=['gold_return_next', 'real_rate_change', 'dxy_change',
                                     'vix', 'yield_spread_change', 'inflation_exp_change'])

nan_after = final_df.isna().sum().sum()
print(f"  NaN after imputation: {nan_after}")
print(f"  Final dataset: {len(final_df)} rows")

assert all(col in final_df.columns for col in FEATURE_COLUMNS), "Missing features after merge!"
assert TARGET in final_df.columns, "Target not found!"
print(f"\nAll {len(FEATURE_COLUMNS)} features present")
print(f"Dataset shape: {final_df.shape}")
print(f"Date range: {final_df.index.min()} to {final_df.index.max()}")

## Train/Val/Test Split (70/15/15)

In [ ]:
# === Train/Val/Test Split (70/15/15, time-series order) ===
n_total = len(final_df)
n_train = int(n_total * 0.70)
n_val = int(n_total * 0.15)

train_df = final_df.iloc[:n_train].copy()
val_df = final_df.iloc[n_train:n_train+n_val].copy()
test_df = final_df.iloc[n_train+n_val:].copy()

print(f"\nData split complete:")
print(f"  Train: {len(train_df)} rows ({len(train_df)/n_total*100:.1f}%) - {train_df.index.min()} to {train_df.index.max()}")
print(f"  Val:   {len(val_df)} rows ({len(val_df)/n_total*100:.1f}%) - {val_df.index.min()} to {val_df.index.max()}")
print(f"  Test:  {len(test_df)} rows ({len(test_df)/n_total*100:.1f}%) - {test_df.index.min()} to {test_df.index.max()}")
print(f"  Total: {n_total} rows")
print(f"  Samples per feature: {n_train / len(FEATURE_COLUMNS):.1f}:1 (train)")

# Verify no data leakage
assert train_df.index.max() < val_df.index.min(), "Train-val overlap detected!"
assert val_df.index.max() < test_df.index.min(), "Val-test overlap detected!"
print(f"\nNo time-series leakage detected")
print("="*60)

# Prepare arrays for training
X_train = train_df[FEATURE_COLUMNS].values
y_train = train_df[TARGET].values

X_val = val_df[FEATURE_COLUMNS].values
y_val = val_df[TARGET].values

X_test = test_df[FEATURE_COLUMNS].values
y_test = test_df[TARGET].values

# Store dates for output
dates_train = train_df.index
dates_val = val_df.index
dates_test = test_df.index

print(f"\nArray shapes:")
print(f"  X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"  X_val:   {X_val.shape}, y_val:   {y_val.shape}")
print(f"  X_test:  {X_test.shape}, y_test:  {y_test.shape}")

## Metric Functions

In [ ]:
def compute_direction_accuracy(y_true, y_pred):
    """Direction accuracy, excluding zeros."""
    mask = (y_true != 0) & (y_pred != 0)
    if mask.sum() == 0:
        return 0.0
    return (np.sign(y_pred[mask]) == np.sign(y_true[mask])).mean()

def compute_mae(y_true, y_pred):
    """Mean Absolute Error."""
    return np.abs(y_pred - y_true).mean()

def compute_sharpe_trade_cost(y_true, y_pred, cost_bps=5.0):
    """Sharpe ratio with position-change cost (5bps per change)."""
    positions = np.sign(y_pred)

    # Strategy returns (position * actual return)
    strategy_returns = positions * y_true / 100.0  # Convert % to decimal

    # Position changes
    position_changes = np.abs(np.diff(positions, prepend=0))
    trade_costs = position_changes * (cost_bps / 10000.0)  # 5bps = 0.0005

    # Net returns
    net_returns = strategy_returns - trade_costs

    # Annualized Sharpe (252 trading days)
    if len(net_returns) < 2 or net_returns.std() == 0:
        return 0.0
    return (net_returns.mean() / net_returns.std()) * np.sqrt(252)

def compute_hcda(y_true, y_pred, threshold_percentile=80):
    """High-confidence direction accuracy (top 20% by |prediction|)."""
    threshold = np.percentile(np.abs(y_pred), threshold_percentile)
    hc_mask = np.abs(y_pred) > threshold

    if hc_mask.sum() == 0:
        return 0.0, 0.0

    coverage = hc_mask.sum() / len(y_pred)
    hc_pred = y_pred[hc_mask]
    hc_actual = y_true[hc_mask]

    mask = (hc_actual != 0) & (hc_pred != 0)
    if mask.sum() == 0:
        return 0.0, coverage

    da = (np.sign(hc_pred[mask]) == np.sign(hc_actual[mask])).mean()
    return da, coverage

def compute_hcda_bootstrap(y_true, y_pred, bootstrap_std, threshold_percentile=80):
    """
    HCDA using bootstrap variance-based confidence.
    High confidence = LOW variance (certain predictions)
    Top 20% by inverse variance: 1 / (1 + std)
    """
    confidence = 1.0 / (1.0 + bootstrap_std)  # Higher confidence when std is low
    threshold = np.percentile(confidence, threshold_percentile)
    hc_mask = confidence > threshold

    if hc_mask.sum() == 0:
        return 0.0, 0.0

    coverage = hc_mask.sum() / len(y_pred)
    hc_pred = y_pred[hc_mask]
    hc_actual = y_true[hc_mask]

    mask = (hc_actual != 0) & (hc_pred != 0)
    if mask.sum() == 0:
        return 0.0, coverage

    da = (np.sign(hc_pred[mask]) == np.sign(hc_actual[mask])).mean()
    return da, coverage

print("Metric functions defined")

## STEP 1: Optuna HPO (100 trials) - Tuned for 28-Feature Space

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("="*60)
print("OPTUNA HPO (100 trials) - Finding best params for 28 features")
print("="*60)

def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "max_depth": trial.suggest_int("max_depth", 2, 4),
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 50),
        "subsample": trial.suggest_float("subsample", 0.60, 0.95),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.35, 0.70),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.3, 8.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 4.0),
        "learning_rate": trial.suggest_float("learning_rate", 0.010, 0.050),
        "n_estimators": 800,
        "early_stopping_rounds": 50,
        "tree_method": "hist",
        "device": "cuda",
        "random_state": 42,
        "verbosity": 0,
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    pred_val = model.predict(X_val)

    da = compute_direction_accuracy(y_val, pred_val)
    sharpe = compute_sharpe_trade_cost(y_val, pred_val)
    mae = compute_mae(y_val, pred_val)

    # Composite: higher is better
    # Sharpe normalized to [-1, 1]: 3.0 maps to 1.0, -3.0 maps to -1.0
    sharpe_norm = min(max(sharpe / 3.0, -1.0), 1.0)
    # MAE normalized: lower MAE = higher score; 0.5 maps to 1.0, 1.5 maps to -1.0
    mae_norm = min(max((1.0 - mae) / 1.0, -1.0), 1.0)
    score = da * 0.6 + sharpe_norm * 0.3 + mae_norm * 0.1

    # Log additional metrics as user attributes
    trial.set_user_attr('val_da', float(da))
    trial.set_user_attr('val_sharpe', float(sharpe))
    trial.set_user_attr('val_mae', float(mae))
    trial.set_user_attr('best_iteration', int(model.best_iteration) if model.best_iteration is not None else 800)

    return score

study = optuna.create_study(
    direction="maximize",
    sampler=TPESampler(seed=42)
)
study.optimize(objective, n_trials=100, show_progress_bar=False)

best_params = study.best_params.copy()
best_params.update({
    "objective": "reg:squarederror",
    "n_estimators": 800,
    "early_stopping_rounds": 50,
    "tree_method": "hist",
    "device": "cuda",
    "random_state": 42,
    "verbosity": 0,
})

print(f"\nOptuna complete: {len(study.trials)} trials")
print(f"Best score: {study.best_value:.4f}")
print(f"\nBest params:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

best_trial = study.best_trial
print(f"\nBest trial validation metrics:")
print(f"  DA:     {best_trial.user_attrs['val_da']*100:.2f}%")
print(f"  Sharpe: {best_trial.user_attrs['val_sharpe']:.4f}")
print(f"  MAE:    {best_trial.user_attrs['val_mae']:.4f}%")
print(f"  Best iteration: {best_trial.user_attrs['best_iteration']}")

# Compare with attempt 7 params
att7_params = {"max_depth": 2, "min_child_weight": 25, "subsample": 0.765,
               "colsample_bytree": 0.450, "reg_lambda": 2.049, "reg_alpha": 1.107,
               "learning_rate": 0.0215}
print(f"\nOptuna best vs Attempt 7 params (optimized for 24 features):")
for k in att7_params:
    att7_v = att7_params[k]
    new_v = study.best_params.get(k, "N/A")
    print(f"  {k}: {att7_v} -> {new_v}")

## STEP 2: Bootstrap Data Subsampling Ensemble (12 models, best Optuna params)

In [ ]:
print("="*60)
print("BOOTSTRAP DATA SUBSAMPLING ENSEMBLE (12 models, best Optuna params)")
print("="*60)

N_ENSEMBLE = 12
BOOTSTRAP_FRAC = 0.80

rng = np.random.RandomState(42)
ensemble_models = []

n_bootstrap = int(BOOTSTRAP_FRAC * len(X_train))
print(f"\nEnsemble config:")
print(f"  Number of models: {N_ENSEMBLE}")
print(f"  Bootstrap fraction: {BOOTSTRAP_FRAC:.0%}")
print(f"  Bootstrap size: {n_bootstrap} samples (from {len(X_train)} total)")
print(f"  XGBoost params: Optuna best (n_estimators=800, early stopping=50)")
print()

for i in range(N_ENSEMBLE):
    seed = 42 + i
    bootstrap_idx = rng.choice(len(X_train), size=n_bootstrap, replace=True)
    X_boot = X_train[bootstrap_idx]
    y_boot = y_train[bootstrap_idx]

    model_params = best_params.copy()
    model_params['random_state'] = seed

    model = xgb.XGBRegressor(**model_params)
    model.fit(X_boot, y_boot, eval_set=[(X_val, y_val)], verbose=False)
    ensemble_models.append(model)
    best_iter = model.best_iteration if model.best_iteration is not None else 800
    print(f"  Model {i+1:2d}/{N_ENSEMBLE}: seed={seed}, best_iteration={best_iter}")

print(f"\nEnsemble training complete: {len(ensemble_models)} models")

## STEP 3: Ensemble Predictions and Bootstrap Variance

In [ ]:
print("="*60)
print("GENERATING ENSEMBLE PREDICTIONS")
print("="*60)

# === Generate predictions from all ensemble models ===
print("\nGenerating predictions from ensemble...")
ensemble_preds_train = np.array([m.predict(X_train) for m in ensemble_models])
ensemble_preds_val = np.array([m.predict(X_val) for m in ensemble_models])
ensemble_preds_test = np.array([m.predict(X_test) for m in ensemble_models])

# Mean prediction (primary prediction)
pred_train = ensemble_preds_train.mean(axis=0)
pred_val = ensemble_preds_val.mean(axis=0)
pred_test = ensemble_preds_test.mean(axis=0)

# Bootstrap variance (confidence)
bootstrap_std_train = np.std(ensemble_preds_train, axis=0)
bootstrap_std_val = np.std(ensemble_preds_val, axis=0)
bootstrap_std_test = np.std(ensemble_preds_test, axis=0)

# Bootstrap confidence = 1 / (1 + std)
bootstrap_conf_train = 1.0 / (1.0 + bootstrap_std_train)
bootstrap_conf_val = 1.0 / (1.0 + bootstrap_std_val)
bootstrap_conf_test = 1.0 / (1.0 + bootstrap_std_test)

# Combine all for full dataset
pred_full = np.concatenate([pred_train, pred_val, pred_test])
dates_full = pd.Index(list(dates_train) + list(dates_val) + list(dates_test))
y_full = np.concatenate([y_train, y_val, y_test])

print(f"\nEnsemble predictions generated:")
print(f"  Train: mean={pred_train.mean():.4f}, std={pred_train.std():.4f}")
print(f"  Val:   mean={pred_val.mean():.4f}, std={pred_val.std():.4f}")
print(f"  Test:  mean={pred_test.mean():.4f}, std={pred_test.std():.4f}")

print(f"\nBootstrap variance statistics (test set):")
print(f"  Std range: [{bootstrap_std_test.min():.4f}, {bootstrap_std_test.max():.4f}]")
print(f"  Std mean:  {bootstrap_std_test.mean():.4f}")
print(f"  Confidence range: [{bootstrap_conf_test.min():.4f}, {bootstrap_conf_test.max():.4f}]")
print(f"  Confidence mean:  {bootstrap_conf_test.mean():.4f}")

# Compute HCDA with BOTH methods
hcda_bootstrap_test, hcda_bootstrap_cov = compute_hcda_bootstrap(y_test, pred_test, bootstrap_std_test)
hcda_pred_test, hcda_pred_cov = compute_hcda(y_test, pred_test)

print(f"\nHCDA comparison (test set):")
print(f"  Bootstrap variance: {hcda_bootstrap_test*100:.2f}% (N={int(hcda_bootstrap_cov*len(y_test))})")
print(f"  |prediction|:       {hcda_pred_test*100:.2f}% (N={int(hcda_pred_cov*len(y_test))})")
print(f"  Improvement:        {(hcda_bootstrap_test - hcda_pred_test)*100:+.2f}pp")

# Select better HCDA method
use_bootstrap_hcda = hcda_bootstrap_test > hcda_pred_test
if use_bootstrap_hcda:
    print(f"\nUsing bootstrap variance for HCDA (better by {(hcda_bootstrap_test - hcda_pred_test)*100:.2f}pp)")
    primary_hcda_method = 'bootstrap'
    primary_hcda_value = hcda_bootstrap_test
else:
    print(f"\nUsing |prediction| for HCDA (better by {(hcda_pred_test - hcda_bootstrap_test)*100:.2f}pp)")
    primary_hcda_method = 'pred'
    primary_hcda_value = hcda_pred_test

## POST-TRAINING STEP 1: OLS Output Scaling

In [ ]:
print("="*60)
print("OLS OUTPUT SCALING")
print("="*60)

# Compute optimal scaling factor from validation set
numerator = np.sum(pred_val * y_val)
denominator = np.sum(pred_val ** 2)
alpha_ols = numerator / denominator if denominator != 0 else 1.0
alpha_ols = np.clip(alpha_ols, 0.5, 10.0)  # Cap at [0.5, 10.0]

print(f"\nOLS scaling factor: {alpha_ols:.2f}")

# Apply scaling to all predictions (for MAE computation only)
scaled_pred_train = pred_train * alpha_ols
scaled_pred_val = pred_val * alpha_ols
scaled_pred_test = pred_test * alpha_ols
scaled_pred_full = pred_full * alpha_ols

# MAE comparison
mae_raw = np.mean(np.abs(pred_test - y_test))
mae_scaled = np.mean(np.abs(scaled_pred_test - y_test))
print(f"\nMAE (raw):    {mae_raw:.4f}%")
print(f"MAE (scaled): {mae_scaled:.4f}%")
print(f"MAE delta:    {mae_scaled - mae_raw:+.4f}%")

# Verify: DA and Sharpe unchanged by scaling
da_raw = compute_direction_accuracy(y_test, pred_test)
da_scaled = compute_direction_accuracy(y_test, scaled_pred_test)
assert abs(da_raw - da_scaled) < 1e-10, "Scaling changed DA!"
print("\nDA and Sharpe: unchanged by scaling (verified)")

# Select better MAE for reporting
use_scaled = mae_scaled < mae_raw
if use_scaled:
    print(f"\nUsing SCALED predictions for MAE (improvement: {mae_raw - mae_scaled:.4f}%)")
else:
    print(f"\nUsing RAW predictions for MAE (scaling degraded by {mae_scaled - mae_raw:.4f}%)")

## Evaluation on All Splits

In [ ]:
print("="*60)
print("FINAL EVALUATION")
print("="*60)

# Compute metrics for all splits
metrics_all = {}
for split_name, y_true, y_pred_raw, y_pred_scaled in [
    ('train', y_train, pred_train, scaled_pred_train),
    ('val', y_val, pred_val, scaled_pred_val),
    ('test', y_test, pred_test, scaled_pred_test),
]:
    da = compute_direction_accuracy(y_true, y_pred_raw)
    mae_raw_split = compute_mae(y_true, y_pred_raw)
    mae_scaled_split = compute_mae(y_true, y_pred_scaled)
    mae = min(mae_raw_split, mae_scaled_split)
    sharpe = compute_sharpe_trade_cost(y_true, y_pred_raw)
    hc_da, hc_coverage = compute_hcda(y_true, y_pred_raw, threshold_percentile=80)

    metrics_all[split_name] = {
        'direction_accuracy': float(da),
        'high_confidence_da': float(hc_da),
        'high_confidence_coverage': float(hc_coverage),
        'mae': float(mae),
        'mae_raw': float(mae_raw_split),
        'mae_scaled': float(mae_scaled_split),
        'sharpe_ratio': float(sharpe),
    }

# Print metrics
for split_name in ['train', 'val', 'test']:
    m = metrics_all[split_name]
    print(f"\n{split_name.upper()}:")
    print(f"  DA:     {m['direction_accuracy']*100:.2f}%")
    print(f"  HCDA:   {m['high_confidence_da']*100:.2f}% (coverage: {m['high_confidence_coverage']*100:.1f}%)")
    print(f"  MAE:    {m['mae']:.4f}% (raw: {m['mae_raw']:.4f}%, scaled: {m['mae_scaled']:.4f}%)")
    print(f"  Sharpe: {m['sharpe_ratio']:.2f}")

# Overfitting analysis
train_test_da_gap = (metrics_all['train']['direction_accuracy'] - metrics_all['test']['direction_accuracy']) * 100
print(f"\nOVERFITTING:")
print(f"  Train-Test DA gap: {train_test_da_gap:.2f}pp (target: <10pp)")

# Target evaluation
test_m = metrics_all['test']
targets_met = [
    test_m['direction_accuracy'] > 0.56,
    primary_hcda_value > 0.60,
    test_m['mae'] < 0.0075,
    test_m['sharpe_ratio'] > 0.8,
]

print(f"\nTARGET STATUS:")
print(f"  DA > 56%:     {'PASS' if targets_met[0] else 'FAIL'} ({test_m['direction_accuracy']*100:.2f}%)")
print(f"  HCDA > 60%:   {'PASS' if targets_met[1] else 'FAIL'} ({primary_hcda_value*100:.2f}% via {primary_hcda_method})")
print(f"  MAE < 0.75%:  {'PASS' if targets_met[2] else 'FAIL'} ({test_m['mae']:.4f}%)")
print(f"  Sharpe > 0.8: {'PASS' if targets_met[3] else 'FAIL'} ({test_m['sharpe_ratio']:.2f})")
print(f"\nTargets passed: {sum(targets_met)}/4")

# Comparison with references
print(f"\nVs Attempt 7 (best overall):")
print(f"  DA:     {test_m['direction_accuracy']*100:.2f}% (Attempt 7: 60.04%, delta: {(test_m['direction_accuracy']-0.6004)*100:+.2f}pp)")
print(f"  HCDA:   {primary_hcda_value*100:.2f}% (Attempt 7: 64.13%, delta: {(primary_hcda_value-0.6413)*100:+.2f}pp)")
print(f"  MAE:    {test_m['mae']:.4f}% (Attempt 7: 0.9429%, delta: {(test_m['mae']-0.9429)*100:+.4f}pp)")
print(f"  Sharpe: {test_m['sharpe_ratio']:.2f} (Attempt 7: 2.4636, delta: {test_m['sharpe_ratio']-2.4636:+.2f})")

print(f"\nVs Attempt 18 (predecessor with fixed params):")
print(f"  DA:     {test_m['direction_accuracy']*100:.2f}% (Attempt 18: 58.30%, delta: {(test_m['direction_accuracy']-0.5830)*100:+.2f}pp)")
print(f"  HCDA:   {primary_hcda_value*100:.2f}% (Attempt 18: 63.04%, delta: {(primary_hcda_value-0.6304)*100:+.2f}pp)")
print(f"  MAE:    {test_m['mae']:.4f}% (Attempt 18: 0.9527%, delta: {(test_m['mae']-0.9527)*100:+.4f}pp)")
print(f"  Sharpe: {test_m['sharpe_ratio']:.2f} (Attempt 18: 1.8624, delta: {test_m['sharpe_ratio']-1.8624:+.2f})")

## Feature Importance Analysis

In [ ]:
print("="*60)
print("FEATURE IMPORTANCE ANALYSIS (ensemble average)")
print("="*60)

# Average feature importance across ensemble models
all_importances = np.array([m.feature_importances_ for m in ensemble_models])
mean_importances = all_importances.mean(axis=0)
std_importances = all_importances.std(axis=0)

feature_ranking = pd.DataFrame({
    'feature': FEATURE_COLUMNS,
    'importance_mean': mean_importances,
    'importance_std': std_importances,
    'importance_pct': mean_importances / mean_importances.sum() * 100
}).sort_values('importance_mean', ascending=False).reset_index(drop=True)

print(f"\nTop 15 features (by mean importance across {N_ENSEMBLE} ensemble models):")
for i, row in feature_ranking.head(15).iterrows():
    print(f"  {i+1:2d}. {row['feature']:35s}: {row['importance_pct']:.2f}%")

# Real rate submodel feature analysis
rr_features = ['rr_level_change_z', 'rr_slope_chg_z', 'rr_curvature_chg_z', 'rr_slope_level_z']
rr_importance_total = feature_ranking[feature_ranking['feature'].isin(rr_features)]['importance_pct'].sum()
print(f"\nReal rate submodel (4 features) total importance: {rr_importance_total:.2f}%")
for feat in rr_features:
    row = feature_ranking[feature_ranking['feature'] == feat]
    rank = row.index[0] + 1
    imp = row['importance_pct'].values[0]
    print(f"  {feat}: rank {rank}/28, {imp:.2f}%")

print(f"\nNaive baseline (always-up):")
naive_always_up_da = (y_test > 0).sum() / len(y_test)
print(f"  Always-up DA: {naive_always_up_da*100:.2f}%")
print(f"  Model vs naive: {(test_m['direction_accuracy'] - naive_always_up_da)*100:+.2f}pp")

## Save Results

In [ ]:
print("="*60)
print("SAVING RESULTS")
print("="*60)

# 1. predictions.csv (full dataset)
split_labels = ['train'] * len(dates_train) + ['val'] * len(dates_val) + ['test'] * len(dates_test)
predictions_df = pd.DataFrame({
    'date': dates_full,
    'split': split_labels,
    'actual': y_full,
    'prediction_raw': pred_full,
    'prediction_scaled': scaled_pred_full,
    'direction_correct': (np.sign(pred_full) == np.sign(y_full)).astype(int),
    'abs_prediction': np.abs(pred_full),
})

# Add high_confidence flags
threshold_80_pred = np.percentile(np.abs(pred_full), 80)
predictions_df['high_confidence_pred'] = (predictions_df['abs_prediction'] > threshold_80_pred).astype(int)

# Bootstrap confidence for full dataset
bootstrap_conf_full = np.concatenate([bootstrap_conf_train, bootstrap_conf_val, bootstrap_conf_test])
threshold_80_bootstrap = np.percentile(bootstrap_conf_full, 80)
predictions_df['bootstrap_confidence'] = bootstrap_conf_full
predictions_df['high_confidence_bootstrap'] = (predictions_df['bootstrap_confidence'] > threshold_80_bootstrap).astype(int)

# Bootstrap std for full dataset
bootstrap_std_full = np.concatenate([bootstrap_std_train, bootstrap_std_val, bootstrap_std_test])
predictions_df['bootstrap_std'] = bootstrap_std_full

predictions_df.to_csv('predictions.csv', index=False)
print("Saved predictions.csv")

# 2. test_predictions.csv
test_predictions_df = predictions_df[predictions_df['split'] == 'test'].copy()
test_predictions_df.to_csv('test_predictions.csv', index=False)
print("Saved test_predictions.csv")

# 3. submodel_output.csv
predictions_df.to_csv('submodel_output.csv', index=False)
print("Saved submodel_output.csv")

# 4. model.json (first ensemble model)
ensemble_models[0].save_model('model.json')
print("Saved model.json (first ensemble model)")

# 5. training_result.json
training_result = {
    'feature': 'meta_model',
    'attempt': 19,
    'timestamp': datetime.now().isoformat(),
    'architecture': 'XGBoost Optuna HPO (100 trials) + Bootstrap Data Subsampling Ensemble (12 models, 80%) + OLS scaling',
    'phase': '3_meta_model',

    'model_config': {
        'n_features': 28,
        'train_samples': len(X_train),
        'val_samples': len(X_val),
        'test_samples': len(X_test),
        'samples_per_feature_ratio': len(X_train) / 28,
        'n_ensemble_models': N_ENSEMBLE,
        'bootstrap_fraction': BOOTSTRAP_FRAC,
    },

    'optuna_study': {
        'n_trials': len(study.trials),
        'best_score': float(study.best_value),
        'best_params': study.best_params,
        'best_trial_val_da': float(study.best_trial.user_attrs['val_da']),
        'best_trial_val_sharpe': float(study.best_trial.user_attrs['val_sharpe']),
        'best_trial_val_mae': float(study.best_trial.user_attrs['val_mae']),
        'search_space': {
            'max_depth': [2, 4],
            'min_child_weight': [10, 50],
            'subsample': [0.60, 0.95],
            'colsample_bytree': [0.35, 0.70],
            'reg_lambda': [0.3, 8.0],
            'reg_alpha': [0.0, 4.0],
            'learning_rate': [0.010, 0.050],
            'n_estimators': 800,
            'early_stopping_rounds': 50,
        },
        'objective_formula': 'DA * 0.6 + Sharpe_norm * 0.3 + MAE_norm * 0.1',
        'top_5_trials': [
            {
                'number': t.number,
                'score': float(t.value),
                'params': t.params,
                'val_da': float(t.user_attrs['val_da']),
                'val_sharpe': float(t.user_attrs['val_sharpe']),
            }
            for t in sorted(study.trials, key=lambda x: x.value if x.value is not None else -999, reverse=True)[:5]
        ],
    },

    'bootstrap_analysis': {
        'n_ensemble_models': N_ENSEMBLE,
        'bootstrap_fraction': BOOTSTRAP_FRAC,
        'bootstrap_std_range_test': [float(bootstrap_std_test.min()), float(bootstrap_std_test.max())],
        'bootstrap_std_mean_test': float(bootstrap_std_test.mean()),
        'bootstrap_conf_range_test': [float(bootstrap_conf_test.min()), float(bootstrap_conf_test.max())],
        'bootstrap_conf_mean_test': float(bootstrap_conf_test.mean()),
        'hcda_bootstrap': float(hcda_bootstrap_test),
        'hcda_pred': float(hcda_pred_test),
        'hcda_improvement_pp': float((hcda_bootstrap_test - hcda_pred_test) * 100),
    },

    'ols_scaling': {
        'alpha_ols': float(alpha_ols),
        'mae_raw': float(mae_raw),
        'mae_scaled': float(mae_scaled),
        'mae_improvement': float(mae_raw - mae_scaled),
    },

    'primary_hcda_method': primary_hcda_method,
    'primary_hcda_value': float(primary_hcda_value),
    'primary_mae': float(min(mae_raw, mae_scaled)),

    'metrics': metrics_all,

    'feature_importance': {
        'top_15': feature_ranking.head(15)[['feature', 'importance_pct']].to_dict('records'),
        'real_rate_submodel_total_pct': float(rr_importance_total),
        'real_rate_submodel_individual': {
            feat: {
                'rank': int(feature_ranking[feature_ranking['feature'] == feat].index[0] + 1),
                'importance_pct': float(feature_ranking[feature_ranking['feature'] == feat]['importance_pct'].values[0]),
            }
            for feat in rr_features
        },
    },

    'target_evaluation': {
        'direction_accuracy': {
            'target': '> 56.0%',
            'actual': f"{test_m['direction_accuracy']*100:.2f}%",
            'gap': f"{(test_m['direction_accuracy'] - 0.56)*100:+.2f}pp",
            'passed': bool(targets_met[0]),
        },
        'high_confidence_da': {
            'target': '> 60.0%',
            'actual': f"{primary_hcda_value*100:.2f}%",
            'gap': f"{(primary_hcda_value - 0.60)*100:+.2f}pp",
            'passed': bool(targets_met[1]),
            'method_used': primary_hcda_method,
        },
        'mae': {
            'target': '< 0.75%',
            'actual': f"{test_m['mae']:.4f}%",
            'gap': f"{(0.0075 - test_m['mae']):.4f}%",
            'passed': bool(targets_met[2]),
        },
        'sharpe_ratio': {
            'target': '> 0.80',
            'actual': f"{test_m['sharpe_ratio']:.2f}",
            'gap': f"{(test_m['sharpe_ratio'] - 0.8):+.2f}",
            'passed': bool(targets_met[3]),
        },
    },

    'targets_passed': sum(targets_met),
    'targets_total': 4,
    'overall_passed': all(targets_met),

    'overfitting_analysis': {
        'train_test_da_gap_pp': float(train_test_da_gap),
        'target_gap_pp': 10.0,
        'overfitting_check': 'PASS' if train_test_da_gap < 10 else 'FAIL',
    },

    'vs_attempt_7': {
        'da_delta_pp': float((test_m['direction_accuracy'] - 0.6004) * 100),
        'hcda_delta_pp': float((primary_hcda_value - 0.6413) * 100),
        'mae_delta': float(test_m['mae'] - 0.9429),
        'sharpe_delta': float(test_m['sharpe_ratio'] - 2.4636),
    },

    'vs_attempt_18': {
        'da_delta_pp': float((test_m['direction_accuracy'] - 0.5830) * 100),
        'hcda_delta_pp': float((primary_hcda_value - 0.6304) * 100),
        'mae_delta': float(test_m['mae'] - 0.9527),
        'sharpe_delta': float(test_m['sharpe_ratio'] - 1.8624),
    },

    'vs_naive': {
        'naive_always_up_da': f"{naive_always_up_da*100:.2f}%",
        'model_vs_naive_pp': float((test_m['direction_accuracy'] - naive_always_up_da) * 100),
    },

    'prediction_characteristics': {
        'mean_raw': float(pred_test.mean()),
        'std_raw': float(pred_test.std()),
        'min_raw': float(pred_test.min()),
        'max_raw': float(pred_test.max()),
        'positive_pct': float((pred_test > 0).sum() / len(pred_test) * 100),
    },
}

with open('training_result.json', 'w') as f:
    json.dump(training_result, f, indent=2, default=str)
print("Saved training_result.json")

print(f"\n{'='*60}")
print("TRAINING COMPLETE")
print(f"{'='*60}")
print(f"Finished: {datetime.now().isoformat()}")
print(f"\nFinal Status:")
print(f"  Attempt: 19")
print(f"  Features: 28")
print(f"  Ensemble: {N_ENSEMBLE} models, {BOOTSTRAP_FRAC:.0%} bootstrap")
print(f"  Optuna trials: {len(study.trials)}")
print(f"  Best Optuna score: {study.best_value:.4f}")
print(f"  OLS alpha: {alpha_ols:.2f}")
print(f"  HCDA method: {primary_hcda_method.upper()}")
print(f"  Targets passed: {sum(targets_met)}/4")
if all(targets_met):
    print(f"  ALL TARGETS MET")
else:
    failed = [t for t, m in zip(['DA', 'HCDA', 'MAE', 'Sharpe'], targets_met) if not m]
    print(f"  Improvements needed on: {failed}")